## Testing Mesh motion of single step static simulation 

RBF shape parametrization technique is based on the definition of a map, $\mathcal{M}(\boldsymbol{x}) : \mathbb{R}^n \rightarrow \mathbb{R}^n, $that allows the possibility of transferring data across non-matching grids and facing the dynamic mesh handling. The map introduced is defines as follows
$$
\mathcal{M}(\boldsymbol{x}) = p(\boldsymbol{x}) + \sum_{i=1}^{\mathcal{N}_C} \gamma_i \varphi(\| \boldsymbol{x} - \boldsymbol{x_{C_i}} \|)
$$
where $p(\boldsymbol{x})$ is a low_degree polynomial term, $\gamma_i$ is the weight, corresponding to the a-priori selected $\mathcal{N}_C $ control points, associated to the i-th basis function, and $\varphi(\| \boldsymbol{x} - \boldsymbol{x_{C_i}} \|)$ a radial function based on the Euclidean distance between the control points position $\boldsymbol{x_{C_i}}$ and $\boldsymbol{x}.$ A radial basis function, generally, is a real-valued function whose value depends only on the distance from the origin, so that $\varphi(\boldsymbol{x}) = \tilde{\varphi}(\| \boldsymbol{x} \|).$

The matrix version of the formula above is:

$\mathcal{M}(\boldsymbol{x}) = \boldsymbol{c} + \boldsymbol{Q}\boldsymbol{x} + \boldsymbol{W^T}\boldsymbol{d}(\boldsymbol{x})$

The idea is that after the computation of the weights and the polynomial terms from the coordinates of the control points before and after the deformation, we can deform all the points of the mesh accordingly. Among the most common used radial basis functions for modelling 2D and 3D shapes, we consider Gaussian splines, Multi-quadratic biharmonic splines, Inverted multi-quadratic biharmonic splines, Thin-plate splines, Beckert and Wendland C^2 basis and Polyharmonic splines all defined and implemented below.

In [1]:
from dolfin import *

In [2]:
def solve_linear_elasticity(mesh, boundaries, d):
    c = Constant(d)

    V = VectorFunctionSpace(mesh, "Lagrange", 1)
    u = TrialFunction(V)
    v = TestFunction(V)

    E, nu = 10.0, 0.3
    mu = E/(2.0*(1.0 + nu))
    lmbda = E*nu/((1.0 + nu)*(1.0 -2.0*nu))
    sigma = 2*mu*sym(grad(u)) + lmbda*tr(grad(u))*Identity(3)
    F = inner(sigma, grad(v))*dx 
    a, L = lhs(F), rhs(F)

    bcs = [DirichletBC(V, Constant((0.0, 0.0, 0.0)), boundaries, 1),
           DirichletBC(V.sub(0), c, boundaries, 2)]

    displacement = Function(V)
    solve(a==L, displacement, bcs)
    return displacement

Mesh update

In [3]:
def update_mesh(mesh, displacement, boundaries):

    new_mesh = Mesh(mesh)
    new_boundaries = MeshFunction("size_t", new_mesh, 2)
    new_boundaries.set_values(boundaries.array())
    ALE.move(new_mesh, displacement)
    return new_mesh, new_boundaries

In [4]:
# Original mesh
mesh = UnitCubeMesh(8, 8, 8)
subdomain1 = CompiledSubDomain("near(x[1], 0)")
subdomain2 = CompiledSubDomain("near(x[1], 1)")
boundaries = MeshFunction("size_t", mesh, 2)
boundaries.set_all(0)
subdomain1.mark(boundaries, 1)
subdomain2.mark(boundaries, 2)
# plot(mesh, title = "Original mesh")

## visualisation in paraview

In [6]:
file1= XDMFFile('results/uu_1.xdmf')
file1.parameters["flush_output"] = True
file1.write(mesh)

In [8]:
# First iteration (accepted)
displacement1 = solve_linear_elasticity(mesh, boundaries, 0.1)
mesh1, boundaries1 = update_mesh(mesh, displacement1, boundaries)
# plot(mesh1, title = "First iteration (accepted)")

file2= XDMFFile('results/uu_2.xdmf')
file1.parameters["flush_output"] = True
file1.write(mesh1)

# # Second iteration
# displacement2 = solve_linear_elasticity(mesh1, boundaries1, 0.5)
# mesh2, boundaries2 = update_mesh(mesh1, displacement2, boundaries1)
# plot(mesh2, title = "Second iteration (rejected)")

# # Something went wrong move mesh back to previous
# displacement3 = Function(displacement2.function_space())
# displacement3.vector()[:] = -1.0*displacement2.vector()
# mesh3, boundaries3 = update_mesh(mesh2, displacement3, boundaries2)
# plot(mesh3, title = "Moved back from second iteration (same as first iteration)")

# from numpy.linalg import norm as np_norm
# print (np_norm(mesh3.coordinates() - mesh1.coordinates()))

# interactive()